# Проект "Анализ базы данных книг стартапа - приложения для чтения"

**Объект исследования** – данные о книгах, их авторах, издательствах и оценках пользователей 

**Цель проекта** – исследование базы данных книг для формирования ценностного предложения приложения для чтения книг по подписке

**Входные данные** — база данных из 5 связанных таблиц (данные о книгах, об авторах, об издательствах,  о пользовательских оценках книг, о пользовательских обзорах на книги)

В процессе работы будут реализованы следующие шаги:

- загрузка и знакомство с данными
- исследование определенных метрик
- выводы по этапам и итоговые выводы

## Доступ к базе данных

In [1]:
# импортируем библиотеки

import pandas as pd
import sqlalchemy as sa

In [2]:
# устанавливаем параметры

db_config = {
'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db' # название базы данных
}
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

In [3]:
# сохраняем коннектор

engine = sa.create_engine(connection_string, connect_args={'sslmode':'require'})

In [4]:
# чтобы выполнить SQL-запрос, пишем функцию с использованием Pandas

def get_sql_data(query:str, engine:sa.engine.base.Engine=engine) -> pd.DataFrame:
    with engine.connect() as con:
        return pd.read_sql(sql=sa.text(query), con = con)

## Загрузка и знакомство с данными

**Books**

In [5]:
# таблица Books - сколько строк в таблице всего

query = '''
SELECT COUNT(*)
FROM books 
'''

get_sql_data(query)

,count
0,1000


In [6]:
# выведем первые 5 строк Books

query = '''
SELECT *
FROM books
LIMIT 5
'''
get_sql_data(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


**Authors**

In [7]:
# таблица Authors - сколько строк в таблице всего
query = '''
SELECT COUNT(*)
FROM authors 
'''

get_sql_data(query)

,count
0,636


In [8]:
# выведем первые 5 строк Authors

query = '''
SELECT *
FROM authors
LIMIT 5
'''
get_sql_data(query)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


**Publishers**

In [9]:
# таблица Publishers - сколько строк в таблице всего
query = '''
SELECT COUNT(*)
FROM publishers
'''

get_sql_data(query)

,count
0,340


In [10]:
# выведем первые 5 строк Publishers

query = '''
SELECT *
FROM publishers
LIMIT 5
'''
get_sql_data(query)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


**Ratings**

In [11]:
# таблица Ratings - сколько строк в таблице всего
query = '''
SELECT COUNT(*)
FROM ratings
'''
get_sql_data(query)

,count
0,6456


In [12]:
# выведем первые 5 строк Ratings

query = '''
SELECT *
FROM ratings
LIMIT 5
'''
get_sql_data(query)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


**Reviews**

In [13]:
# таблица Reviews - сколько строк в таблице всего
query = '''
SELECT COUNT(*)
FROM reviews
'''

get_sql_data(query)

,count
0,2793


In [14]:
# выведем первые 5 строк Reviews

query = '''
SELECT *
FROM reviews
LIMIT 5
'''
get_sql_data(query)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Все данные загружены, понятны названия столцов, сопоставимы столбцы.

## Задания

### Сколько книг вышло после 1 января 2000 года?

In [15]:
query = '''
SELECT COUNT(book_id)
FROM books
WHERE publication_date > '2000-01-01'
LIMIT 5
'''
get_sql_data(query)

,count
0,819


После 1 января 2000 года вышло 819 книг.

### Для каждой книги посчитайте количество обзоров и среднюю оценку

In [16]:
query = '''
SELECT b.book_id,
       title,
       COUNT(DISTINCT rev.review_id) rev_cnt,
       AVG(rating) avg_rat
FROM books b
LEFT JOIN ratings rat ON b.book_id = rat.book_id
LEFT JOIN reviews rev ON b.book_id = rev.book_id
GROUP BY b.book_id
ORDER BY avg_rat DESC
'''
get_sql_data(query)

,book_id,title,rev_cnt,avg_rat
0,86,Arrows of the Queen (Heralds of Valdemar #1),2,5.00
1,901,The Walking Dead Book One (The Walking Dead #...,2,5.00
2,390,Light in August,2,5.00
3,972,Wherever You Go There You Are: Mindfulness Me...,2,5.00
4,136,Captivating: Unveiling the Mystery of a Woman'...,2,5.00
...,...,...,...,...
995,915,The World Is Flat: A Brief History of the Twen...,3,2.25
996,316,His Excellency: George Washington,2,2.00
997,202,Drowning Ruth,3,2.00
998,371,Junky,2,2.00


Средний рейтинг варьируется от 1,5 до 5. Самый высокий средний рейтинг у книги Arrows of the Queen (Heralds of Valdemar #1)	 - **5**

### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц (исключение брошюр)

In [17]:
query = '''
SELECT publisher
FROM publishers p
JOIN books b ON p.publisher_id = b.publisher_id
WHERE num_pages > 50
GROUP BY publisher
ORDER BY COUNT (book_id) DESC
LIMIT 1
'''
get_sql_data(query)

,publisher
0,Penguin Books


Издательство Penguin Books выпустило больше всего книг.

### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [18]:
query = '''
WITH book_rat AS (SELECT b.book_id,
                         author_id,
                         COUNT(rating),
                         AVG(rating) avg_rat
                FROM books b
                LEFT JOIN ratings rat ON b.book_id = rat.book_id
                GROUP BY b.book_id
                HAVING COUNT(rating)>50)
SELECT author
FROM authors a
RIGHT JOIN book_rat ON book_rat.author_id = a.author_id
GROUP  BY author
ORDER BY AVG(avg_rat) DESC
LIMIT 1

'''
get_sql_data(query)

,author
0,J.K. Rowling/Mary GrandPré


Автор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré.

### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [19]:
query = '''
WITH t1 AS (SELECT DISTINCT username,
                   COUNT(rating) rat_cnt
            FROM ratings rat
            GROUP BY DISTINCT username
            HAVING COUNT(rating) > 48),

t2 AS (SELECT DISTINCT username,
                   COUNT(text) rev_cnt
            FROM reviews rev
            GROUP BY DISTINCT username)
SELECT AVG(rev_cnt)
FROM t1
LEFT JOIN t2 ON t1.username = t2.username


'''
get_sql_data(query)

,avg
0,24.0


Среднее количество обзоров пользователей, которые поставили более 48 оценок, составляет **24**

### Выводы по исследованию

Проведен анализ базы данных сервиса для чтения книг по подписке. По результатам анализа сделаны следующие выводы:
- После 1 января 2000 годас вышло **819** книг
- Средний рейтинг варьируется от 1,5 до 5. Самый **высокий средний рейтинг** у книги **Arrows of the Queen (Heralds of Valdemar #1)**, составляет **5**
- Больше всего книг выпустило издательство **Penguin Books**
- Автор с самой высокой средней оценкой книг - **J.K. Rowling/Mary GrandPré**
- **Среднее количество обзоров** активных пользователей(которые поставили более 48 оценок) составляет **24**
